# Complete MRI T1-T2 CycleGAN Implementation

**Full 114+ Million Parameter Model**

This notebook implements the COMPLETE CycleGAN architecture with:
- Full U-Net generators (54M parameters each)
- Full PatchGAN discriminators (2.8M parameters each)
- Professional training pipeline
- Real-time monitoring and visualization

## Quick Start
1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Run all cells**: Runtime → Run all
3. **Upload dataset** when prompted
4. **Monitor training** with real-time visualizations

---

## 1. Environment Setup
Setting up the complete environment for professional CycleGAN training.

In [1]:
# Environment setup for complete CycleGAN
import os
import sys

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running in local environment")

# Install required packages
if IN_COLAB:
    !pip install -q opencv-python-headless
    print("Additional packages installed")

# Import all required libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import cv2
from tqdm import tqdm
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")

# GPU setup and optimization
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth to prevent OOM
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU available: {gpus[0].name}")
        print(f"Memory growth enabled for optimal performance")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print("No GPU found. Training will be very slow.")
    print("Enable GPU: Runtime → Change runtime type → Hardware accelerator → GPU")

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("Environment setup complete - Ready for full CycleGAN training!")

Running in Google Colab
Additional packages installed
TensorFlow version: 2.18.0
GPU available: /physical_device:GPU:0
Memory growth enabled for optimal performance
Environment setup complete - Ready for full CycleGAN training!


## 2. Dataset Preparation
Upload and prepare the MRI dataset for professional training.

In [ ]:
# Dataset upload and extraction
if IN_COLAB:
    from google.colab import files

    print("Please upload your MRI+T1_T2+Dataset.RAR file")
    print("(File size: ~100MB, upload may take 2-3 minutes)")

    uploaded = files.upload()

    # Find and extract RAR file
    rar_file = None
    for filename in uploaded.keys():
        if filename.endswith(('.RAR', '.rar')):
            rar_file = filename
            break

    if rar_file:
        print(f"Found dataset: {rar_file}")

        # Install and use unrar
        !apt-get update -qq && apt-get install -qq unrar
        !unrar x "{rar_file}" -y

        print("Dataset extracted successfully")
    else:
        print("No RAR file found. Please upload the MRI dataset.")
        raise FileNotFoundError("Dataset file not found")
else:
    print("Local environment - ensure dataset is in current directory")

# Verify and analyze dataset
t1_path = "Tr1/TrainT1"
t2_path = "Tr2/TrainT2"

if os.path.exists(t1_path) and os.path.exists(t2_path):
    t1_files = sorted([f for f in os.listdir(t1_path) if f.endswith('.png')])
    t2_files = sorted([f for f in os.listdir(t2_path) if f.endswith('.png')])

    print(f"Dataset Analysis:")
    print(f"   T1 Images: {len(t1_files)}")
    print(f"   T2 Images: {len(t2_files)}")
    print(f"   Total Images: {len(t1_files) + len(t2_files)}")

    # Display sample images
    if t1_files and t2_files:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        # Show multiple T1 samples
        for i in range(3):
            if i < len(t1_files):
                img = Image.open(os.path.join(t1_path, t1_files[i]))
                axes[0, i].imshow(img, cmap='gray')
                axes[0, i].set_title(f'T1 Sample {i+1}')
                axes[0, i].axis('off')

        # Show multiple T2 samples
        for i in range(3):
            if i < len(t2_files):
                img = Image.open(os.path.join(t2_path, t2_files[i]))
                axes[1, i].imshow(img, cmap='gray')
                axes[1, i].set_title(f'T2 Sample {i+1}')
                axes[1, i].axis('off')

        plt.suptitle('MRI Dataset Samples - Ready for CycleGAN Training', fontsize=16)
        plt.tight_layout()
        plt.show()

        # Image statistics
        sample_t1 = Image.open(os.path.join(t1_path, t1_files[0]))
        sample_t2 = Image.open(os.path.join(t2_path, t2_files[0]))

        print(f"Image Properties:")
        print(f"   T1 size: {sample_t1.size}")
        print(f"   T2 size: {sample_t2.size}")
        print(f"   Format: {sample_t1.mode}")

        print("Dataset ready for professional CycleGAN training")
else:
    print("Dataset structure not found")
    print("Expected structure:")
    print("  Tr1/TrainT1/  # T1 weighted images")
    print("  Tr2/TrainT2/  # T2 weighted images")
    raise FileNotFoundError("Dataset structure not found")

Please upload your MRI+T1_T2+Dataset.RAR file
(File size: ~100MB, upload may take 2-3 minutes)


## 3. Complete CycleGAN Architecture
Implementing the full professional CycleGAN with 114+ million parameters.

In [ ]:
# Instance Normalization Layer - Critical for CycleGAN
class InstanceNormalization(tf.keras.layers.Layer):
    """
    Instance Normalization layer optimized for CycleGAN style transfer.
    Essential for maintaining image-specific statistics during transformation.
    """

    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(
            name='scale',
            shape=input_shape[-1:],
            initializer=tf.random_normal_initializer(1., 0.02),
            trainable=True
        )
        self.offset = self.add_weight(
            name='offset',
            shape=input_shape[-1:],
            initializer='zeros',
            trainable=True
        )
        super(InstanceNormalization, self).build(input_shape)

    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset

    def get_config(self):
        config = super(InstanceNormalization, self).get_config()
        config.update({'epsilon': self.epsilon})
        return config

print("Instance Normalization layer implemented")

In [ ]:
# Complete Generator Building Blocks
def downsample(filters, size, norm_type='instancenorm', apply_norm=True):
    """Downsampling block for U-Net encoder"""
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2D(
        filters, size, strides=2, padding='same',
        kernel_initializer=initializer, use_bias=False
    ))

    if apply_norm:
        if norm_type.lower() == 'batchnorm':
            result.add(tf.keras.layers.BatchNormalization())
        elif norm_type.lower() == 'instancenorm':
            result.add(InstanceNormalization())

    result.add(tf.keras.layers.LeakyReLU())
    return result

def upsample(filters, size, norm_type='instancenorm', apply_dropout=False):
    """Upsampling block for U-Net decoder"""
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2DTranspose(
        filters, size, strides=2, padding='same',
        kernel_initializer=initializer, use_bias=False
    ))

    if norm_type.lower() == 'batchnorm':
        result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
        result.add(InstanceNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())
    return result

print("Generator building blocks implemented")

In [ ]:
# Complete U-Net Generator - Full 54M Parameter Architecture
def Generator(norm_type='instancenorm', target=None):
    """
    Complete U-Net Generator for CycleGAN
    Architecture: 54+ million parameters
    Input: 256x256x1 → Output: 256x256x1
    """
    inputs = tf.keras.layers.Input(shape=[256, 256, 1])

    # Encoder (Downsampling) - 8 levels
    down_stack = [
        downsample(64, 4, norm_type, apply_norm=False),   # (bs, 128, 128, 64)
        downsample(128, 4, norm_type),                    # (bs, 64, 64, 128)
        downsample(256, 4, norm_type),                    # (bs, 32, 32, 256)
        downsample(512, 4, norm_type),                    # (bs, 16, 16, 512)
        downsample(512, 4, norm_type),                    # (bs, 8, 8, 512)
        downsample(512, 4, norm_type),                    # (bs, 4, 4, 512)
        downsample(512, 4, norm_type),                    # (bs, 2, 2, 512)
        downsample(512, 4, norm_type),                    # (bs, 1, 1, 512)
    ]

    # Decoder (Upsampling) - 8 levels with skip connections
    up_stack = [
        upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 2, 2, 1024)
        upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 4, 4, 1024)
        upsample(512, 4, norm_type, apply_dropout=True),  # (bs, 8, 8, 1024)
        upsample(512, 4, norm_type),                      # (bs, 16, 16, 1024)
        upsample(256, 4, norm_type),                      # (bs, 32, 32, 512)
        upsample(128, 4, norm_type),                      # (bs, 64, 64, 256)
        upsample(64, 4, norm_type),                       # (bs, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(
        1, 4, strides=2, padding='same',
        kernel_initializer=initializer,
        activation='tanh'  # Output range [-1, 1]
    )  # (bs, 256, 256, 1)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    model_name = f"generator_G" if target is None else f"generator_{target}"
    return tf.keras.Model(inputs=inputs, outputs=x, name=model_name)

print("Complete U-Net Generator implemented (54M parameters)")

In [ ]:
# Complete PatchGAN Discriminator
def Discriminator(norm_type='instancenorm', target=None):
    """
    Complete PatchGAN Discriminator for CycleGAN
    Architecture: 2.8+ million parameters
    Output: 30x30 patch classifications
    """
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[256, 256, 1], name='input_image')

    x = inp

    # Layer 1: No normalization
    down1 = downsample(64, 4, norm_type, False)(x)  # (bs, 128, 128, 64)

    # Layer 2
    down2 = downsample(128, 4, norm_type)(down1)    # (bs, 64, 64, 128)

    # Layer 3
    down3 = downsample(256, 4, norm_type)(down2)    # (bs, 32, 32, 256)

    # Layer 4: Stride 1
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(
        512, 4, strides=1,
        kernel_initializer=initializer,
        use_bias=False
    )(zero_pad1)  # (bs, 31, 31, 512)

    if norm_type.lower() == 'batchnorm':
        norm1 = tf.keras.layers.BatchNormalization()(conv)
    elif norm_type.lower() == 'instancenorm':
        norm1 = InstanceNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(norm1)

    # Final layer
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)

    last = tf.keras.layers.Conv2D(
        1, 4, strides=1,
        kernel_initializer=initializer
    )(zero_pad2)  # (bs, 30, 30, 1)

    model_name = f"discriminator_D" if target is None else f"discriminator_{target}"
    return tf.keras.Model(inputs=inp, outputs=last, name=model_name)

print("Complete PatchGAN Discriminator implemented (2.8M parameters)")

In [ ]:
# Complete Loss Functions for CycleGAN
def generator_loss(fake):
    """Generator loss - encourages realistic generation"""
    return tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=fake, labels=tf.ones_like(fake)
        )
    )

def discriminator_loss(real, fake):
    """Discriminator loss - distinguishes real from fake"""
    real_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=real, labels=tf.ones_like(real)
        )
    )

    fake_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=fake, labels=tf.zeros_like(fake)
        )
    )

    total_loss = real_loss + fake_loss
    return total_loss * 0.5

def calc_cycle_loss(real_image, cycled_image, lambda_weight):
    """Cycle consistency loss - preserves content"""
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return lambda_weight * loss1

def identity_loss(real_image, same_image, lambda_weight):
    """Identity loss - preserves color composition"""
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return lambda_weight * 0.5 * loss

print("Complete loss functions implemented")

In [ ]:
# Create Complete CycleGAN Models
print("Creating complete CycleGAN architecture...")
print("This is the FULL professional implementation with 114+ million parameters")

# Create generators (54M parameters each)
generator_g = Generator(norm_type='instancenorm', target='T1_to_T2')  # T1 -> T2
generator_f = Generator(norm_type='instancenorm', target='T2_to_T1')  # T2 -> T1

# Create discriminators (2.8M parameters each)
discriminator_x = Discriminator(norm_type='instancenorm', target='T1')  # T1 discriminator
discriminator_y = Discriminator(norm_type='instancenorm', target='T2')  # T2 discriminator

# Create optimizers with CycleGAN-optimized settings
generator_g_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_x_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

# Display model statistics
print(f"\nComplete CycleGAN Architecture Statistics:")
print(f"   Generator G (T1→T2): {generator_g.count_params():,} parameters")
print(f"   Generator F (T2→T1): {generator_f.count_params():,} parameters")
print(f"   Discriminator X (T1): {discriminator_x.count_params():,} parameters")
print(f"   Discriminator Y (T2): {discriminator_y.count_params():,} parameters")

total_params = (generator_g.count_params() + generator_f.count_params() +
                discriminator_x.count_params() + discriminator_y.count_params())

print(f"   TOTAL PARAMETERS: {total_params:,}")
print(f"   Model Size: ~{total_params * 4 / (1024**2):.1f} MB")

if total_params > 100_000_000:
    print("SUCCESS: Complete 114+ million parameter CycleGAN created!")
else:
    print("ERROR: Model is smaller than expected")

print("\nReady for professional CycleGAN training!")